In [17]:
from traceback import format_exc
from typing import Annotated, Literal

from langgraph.graph import END, MessagesState, START, StateGraph
from langgraph.prebuilt import create_react_agent
from langgraph.types import Command
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.tools import tool
from langchain_experimental.utilities import PythonREPL
from langchain_openai import ChatOpenAI
from langchain_tavily import TavilySearch
from IPython.display import Image, display

### Tools

In [3]:
tavily_tool = TavilySearch(max_results=5)


In [100]:
repl = PythonREPL()

@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    result_str = f"Successfully executed:\n```python\n{code}\n```\nStdout: {result}"
    return (
        result_str + "\n\nIf you have completed all tasks, respond with FINAL ANSWER."
    )

### Graph

In [101]:
def make_system_prompt(suffix: str) -> str:
    return (
        "You are a helpful AI assistant, collaborating with other assistants."
        " Use the provided tools to progress towards answering the question."
        " If you are unable to fully answer, that's OK, another assistant with different tools "
        " will help where you left off. Execute what you can to make progress."
        
        "\nRespones format:"
        "- Return the response in json dict format, with key `response` and `node`, the `response` will have your answer and `node` will contain your colleague name"
        "- Prefix your answer with FINAL ANSWER so the team knows to stop, include the FINAL ANSWER in the value of `response` key. If you answer is complete, then set `node` key to None"
        " If you or any of the other assistants have the final answer or deliverable, include \"FINAL ANSWER\" before your response"
        "- Example for response with FINAL ANSWER: {'response': 'FINAL ANSWER <REPLACE THIS WITH YOUR ANSWER>', 'node': 'None'}"
        "- Example for response without FINAL ANSWER: {'response': '<REPLACE THIS WITH YOUR ANSWER>', 'node': '<REPLACE THIS WITH YOUR COLLEAGUE NAME>'}"
        
        f"\n{suffix}"
    )

In [102]:
llm = ChatOpenAI(model='gpt-4o')

In [103]:
def get_next_node(last_message: BaseMessage, goto: str):
    print("Last message:", last_message.content)
    # if "FINAL ANSWER" in last_message.content:
    #     # Any agent decided the work is done
    #     return END
    # return goto
    return END

In [104]:
# Research agent and node
research_agent = create_react_agent(
    llm,
    tools=[tavily_tool],
    prompt=make_system_prompt(
        "You can only do research. You are working with a chart generator colleague and travel planner colleague."
        "\nIf the query is regarding the traveling, don't process the response, just provided the raw response to your colleague travel planner."
        
    ),
)



def research_node(
    state: MessagesState,
) -> Command[Literal['chart_generator', END]]:
    result = research_agent.invoke(state)
    goto = get_next_node(result['messages'][-1], 'chart_generator')
    # wrap in a human message, as not all providers allow
    # AI message at the last position of the input messages list
    result['messages'][-1] = HumanMessage(
        content=result['messages'][-1].content, name='researcher'
    )
    print("Research node:", result['messages'][-1].content)
    return Command(
        update={
            # share internal message history of research agent with other agents
            "messages": result['messages'],
        },
        goto=goto
    )

In [105]:
# Chart generator agent and node
# NOTE: THIS PERFORMS ARBITRARY CODE EXECUTION, WHICH CAN BE UNSAFE WHEN NOT SANDBOXED
chart_agent = create_react_agent(
    llm,
    [python_repl_tool],
    prompt=make_system_prompt(
        "You can only generate charts. You are working with researcher colleague"
    ),
)

def chart_node(state: MessagesState) -> Command[Literal['researcher', END]]:
    result = chart_agent.invoke(state)
    goto = get_next_node(result['messages'][-1], 'researcher')
    # wrap in a human message, as not all providers allow
    # AI message at the last position of the input messages list
    result['messages'][-1] = HumanMessage(
        content=result['messages'][-1].content, name='chart_generator'
    )
    print("Chart node:", result['messages'][-1].content)
    return Command(
        update={
            # shared internal message history of chart agent with other agents
            "messages": result['messages']
        },
        goto=goto,
    )

In [114]:
# Traver platter
travel_agent = create_react_agent(
    llm,
    tools=[],
    prompt=make_system_prompt(
        "You can only do travel plan. You are working with researcher colleague, who will help in doing the research."
        " You job is the create detailed plans using the details provided."
    ),
)

def travel_node(state: MessagesState) -> Command[Literal['researcher', END]]:
    result = travel_agent.invoke(state)
    goto = get_next_node(result['messages'][-1], 'researcher')
    # wrap in a human message, as not all providers allow
    # AI message at the last position of the input messages list
    result['messages'][-1] = HumanMessage(
        content=result['messages'][-1].content, name='chart_generator'
    )
    print("Travel node:", result['messages'][-1].content)
    return Command(
        update={
            # shared internal message history of chart agent with other agents
            "messages": result['messages']
        },
        goto=goto,
    )

In [115]:
# travel_research = """Based on the search results, here are some recommendations for travel in each continent:\n\n### Europe\n- **Amsterdam, Netherlands**: A vibrant city with canals, museums, and bustling nightlife. Visit museums, take a canal tour, and explore the city by bike.\n- **Barcelona, Spain**: Known for its architecture by Gaudí, beautiful beaches, and vibrant neighborhoods.\n- **Berlin, Germany**: Rich in history, art, and culture. Explore museums, historical sites, and experience the nightlife.\n\n### Asia\n- **Tokyo, Japan**: A blend of modern and traditional, offering shopping, cuisine, and historical temples.\n- **Palawan, Philippines**: Known for its stunning beaches, clear waters, and excellent diving spots.\n- **Bangkok, Thailand**: Vibrant city with temples, markets, and delicious street food.\n\n### South America\n- **Cusco, Peru**: Gateway to Machu Picchu and rich in Inca history.\n- **Rio de Janeiro, Brazil**: Famous for its beaches, carnivals, and the iconic Christ the Redeemer statue.\n- **Buenos Aires, Argentina**: Known for tango, vibrant neighborhoods, and culinary delights.\n\n### Australia\n- **Sydney**: Famous for the Sydney Opera House, beaches, and vibrant cultural scene.\n- **Great Barrier Reef**: A must-see for snorkeling and diving enthusiasts.\n- **Melbourne**: Known for its art, coffee culture, and diverse cuisine.\n\n### Africa\n- **Namibia**: Explore the Namib Desert and national parks with unique landscapes.\n- **Botswana**: Known for safaris and wildlife in places like the Okavango Delta.\n- **Morocco**: Experience the culture, food, and historic sites in cities like Marrakesh and Fes.\n\nI suggest collaborating with the travel planner colleague for a detailed itinerary, including length of stay and accommodations in each destination."""
travel_research = """Staying at each of the given quiet destinations, here are some detailed plans and top accommodations for your travel:\n\n**1. Baja California Sur, Mexico**\n- **Places to Visit:** \n  - Cabo Pulmo for scuba diving\n  - La Ventana for mobula viewing\n  - Ca\u00f1on de la Zorra for hiking\n  - Loreto - noted for historic architecture.\n  - Mulege - famous for beaches and unique spots.\n  - Espiritu Santo Island from La Paz is perfect for marine wildlife.\n- **Accommodation Options:**\n  - Paradisus Los Cabos - Adults only\n  - Hotel Riu Palace Cabo San Lucas\n  - Villa La Valencia Beach Resort & Spa\n\n**2. Bulgaria**\n- **Places to Visit:** \n  - St. Alexander Nevski Cathedral\n  - Old Nessebar\n  - Vitosha Boulevard\n  - Rila mountains and monasteries\n  - Plovdiv and Veliko Tarnovo for historical sites.\n- **Accommodation Options:**\n  - HI Hotels Imperial Resort\n  - Meliá Sunny Beach\n  - Alua Helios Bay for all-inclusive luxury experience.\n\n**3. St John, U.S. Virgin Islands**\n- **Places to Visit:** \n  - Virgin Islands National Park\n  - Annaberg Sugar Mill Ruins\n  - Trunk Bay for snorkeling\n  - Waterlemon Cay for hidden gems\n  - Jeep ride adventure\n- **Accommodation Options:**\n  - Trunk Bay beachside stays\n  - Several luxurious camping options offer on-site amenities like rentals for snorkeling.\n\n**4. Faroe Islands**\n- **Places to Visit:** \n  - Waterfall at Múlafossur\n  - Kalsoy island for Kallur lighthouse\n  - Scenic buttercup routes\n- **Accommodation Options:**\n  - Hilton Garden Inn Faroe Islands\n  - Hotel Føroyar\n  - Quaint cottages at Mulafossur\n\n**5. Sifnos, Greece**\n- **Places to Visit:** \n  - Explore Kastro for a journey into the past\n  - Vathi for a calm beachside day\n  - Chrissopigi monastery\n  - Apollonia for the village experience.\n- **Accommodation Options:**\n  - Platis Gialos Beach area\n  - Nos Hotel & Villas\n\nFor a 4-month trip, it’s ideal to spend approximately 3-4 weeks in each destination allowing time to explore, relax, and experience local culture thoroughly. Planning such a trip would involve careful consideration of seasons and local events. Consider checking visa requirements and booking accommodations that allow flexible adjustments.\n\nFINAL ANSWER: For your query regarding a detailed 4-month itinerary visit combining quiet destinations Baja California Sur, Mexico; Bulgaria; St. John, U.S. Virgin Islands; Faroe Islands; and Sifnos, Greece, outlined are key places to visit and accommodation options to consider in each location. Use this detailed itinerary and plan to experience a unique and tranquil travel adventure."""
travel_node({"messages": travel_research})

Last message: {'response': 'FINAL ANSWER: For your query regarding a detailed 4-month itinerary visit combining quiet destinations Baja California Sur, Mexico; Bulgaria; St. John, U.S. Virgin Islands; Faroe Islands; and Sifnos, Greece, outlined are key places to visit and accommodation options to consider in each location. Use this detailed itinerary and plan to experience a unique and tranquil travel adventure.', 'node': None}
Travel node: {'response': 'FINAL ANSWER: For your query regarding a detailed 4-month itinerary visit combining quiet destinations Baja California Sur, Mexico; Bulgaria; St. John, U.S. Virgin Islands; Faroe Islands; and Sifnos, Greece, outlined are key places to visit and accommodation options to consider in each location. Use this detailed itinerary and plan to experience a unique and tranquil travel adventure.', 'node': None}


Command(update={'messages': [HumanMessage(content='Staying at each of the given quiet destinations, here are some detailed plans and top accommodations for your travel:\n\n**1. Baja California Sur, Mexico**\n- **Places to Visit:** \n  - Cabo Pulmo for scuba diving\n  - La Ventana for mobula viewing\n  - Cañon de la Zorra for hiking\n  - Loreto - noted for historic architecture.\n  - Mulege - famous for beaches and unique spots.\n  - Espiritu Santo Island from La Paz is perfect for marine wildlife.\n- **Accommodation Options:**\n  - Paradisus Los Cabos - Adults only\n  - Hotel Riu Palace Cabo San Lucas\n  - Villa La Valencia Beach Resort & Spa\n\n**2. Bulgaria**\n- **Places to Visit:** \n  - St. Alexander Nevski Cathedral\n  - Old Nessebar\n  - Vitosha Boulevard\n  - Rila mountains and monasteries\n  - Plovdiv and Veliko Tarnovo for historical sites.\n- **Accommodation Options:**\n  - HI Hotels Imperial Resort\n  - Meliá Sunny Beach\n  - Alua Helios Bay for all-inclusive luxury experien

### Define the Graph

In [107]:
workflow = StateGraph(MessagesState)
workflow.add_node("researcher", research_node)
workflow.add_node("chart_generator", chart_node)
workflow.add_node("travel_planner", travel_node)

workflow.add_edge(START, "researcher")
graph = workflow.compile()

In [108]:
try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    print(format_exc())

Traceback (most recent call last):
  File "/tmp/ipykernel_122879/2657203294.py", line 2, in <module>
    display(Image(graph.get_graph().draw_mermaid_png()))
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/lap-68/Documents/gt-lef/agents/venv/agents/lib/python3.12/site-packages/langchain_core/runnables/graph.py", line 702, in draw_mermaid_png
    return draw_mermaid_png(
           ^^^^^^^^^^^^^^^^^
  File "/home/lap-68/Documents/gt-lef/agents/venv/agents/lib/python3.12/site-packages/langchain_core/runnables/graph_mermaid.py", line 310, in draw_mermaid_png
    img_bytes = _render_mermaid_using_api(
                ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/lap-68/Documents/gt-lef/agents/venv/agents/lib/python3.12/site-packages/langchain_core/runnables/graph_mermaid.py", line 463, in _render_mermaid_using_api
    raise ValueError(msg)
ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 400.

To resolve this issue:
1. Che

### Invoke

In [112]:
events = graph.stream(
    {
        "messages": [
            (
                "user",
                # "First, get the UK's GDP over the past 5 years, then make a line chat of it"
                # "One you make the chart, finish."
                "One want to visit five countries in 4 months, "
                "one want to visit quite countires "
                "generate a details plan for visiting places "
                "and staying in each countries, and during I should at each place."
            ),
            
        ]
    },
    # Maximum number of steps to take in the graph
    {"recursion_limit": 150}
)
for s in events:
    print(s)
    print("-----")

Last message: {'response': 'Staying at each of the given quiet destinations, here are some detailed plans and top accommodations for your travel:\n\n**1. Baja California Sur, Mexico**\n- **Places to Visit:** \n  - Cabo Pulmo for scuba diving\n  - La Ventana for mobula viewing\n  - Ca\u00f1on de la Zorra for hiking\n  - Loreto - noted for historic architecture.\n  - Mulege - famous for beaches and unique spots.\n  - Espiritu Santo Island from La Paz is perfect for marine wildlife.\n- **Accommodation Options:**\n  - Paradisus Los Cabos - Adults only\n  - Hotel Riu Palace Cabo San Lucas\n  - Villa La Valencia Beach Resort & Spa\n\n**2. Bulgaria**\n- **Places to Visit:** \n  - St. Alexander Nevski Cathedral\n  - Old Nessebar\n  - Vitosha Boulevard\n  - Rila mountains and monasteries\n  - Plovdiv and Veliko Tarnovo for historical sites.\n- **Accommodation Options:**\n  - HI Hotels Imperial Resort\n  - Meliá Sunny Beach\n  - Alua Helios Bay for all-inclusive luxury experience.\n\n**3. St Joh